## Підготовка середовища та обробка тексту
Для аналізу тексту ми використаємо TF-IDF (Term Frequency-Inverse Document Frequency) — статистичну міру, що оцінює важливість слова в документі.
Математично вона виражається як:

In [19]:
import pandas as pd
import numpy as np

file_path = '../data/raw/publichna-informatsiia-vikhidni-za-gruden-2025.xlsx'

try:
    df = pd.read_excel(file_path)
    print("Файл успішно завантажено!")
except Exception as e:
    print(f"Помилка при читанні: {e}")


Файл успішно завантажено!


## Побудова моделі кластеризації

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

df['Короткий зміст'] = df['Короткий зміст'].fillna('немає змісту').astype(str)

vectorizer = TfidfVectorizer(max_features=500, stop_words=None) 
X = vectorizer.fit_transform(df['Короткий зміст'])

num_clusters = 4  
kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init=10)
df['Група_документа'] = kmeans.fit_predict(X)

print("Розподіл документів за групами:")
print(df['Група_документа'].value_counts())

order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names_out()

for i in range(num_clusters):
    print(f"\nГрупа {i}: ", end='')
    for ind in order_centroids[i, :5]:
        print(f"'{terms[ind]}'", end=' ')

Розподіл документів за групами:
Група_документа
3    212
2     41
1     22
0     15
Name: count, dtype: int64

Група 0: 'реєстрацією' 'представляти' 'мвс' 'лихошваю' 'постановкою' 
Група 1: 'належністю' 'гр' 'звернення' 'надсилається' 'за' 
Група 2: 'спецперевірки' 'васьківської' 'лист' 'щодо' 'проведення' 
Група 3: 'про' 'щодо' '2025' 'та' 'україни' 

## Пошук аномалій

In [21]:
distances = kmeans.transform(X)
min_distances = np.min(distances, axis=1)

anomaly_idx = np.argsort(min_distances)[-5:]
print("Найбільш специфічні документи за змістом:")
print(df.iloc[anomaly_idx][['Короткий зміст', 'Адресати']])

Найбільш специфічні документи за змістом:
                                        Короткий зміст  \
287  Відповідь на звернення Ніколаєва Ю. щодо сприя...   
8    Щодо повернення працівниками бюджетних установ...   
215  Щодо розповсюдження книжковим інтернет-магазин...   
157  Щодо підвищення рівня кіберзахисту системи еле...   
85   Надається Протокол засідання тимчасової робочо...   

                                              Адресати  
287                                      Ніколаєв Ю.М.  
8    Міністерство економіки, довкілля та сільського...  
215       Адміністрації\nінтернет-магазину\n«TopBooks»  
157  Апарат Ради національної безпеки і оборони Укр...  
85   Міністерство культури та стратегічних комунікацій  


## Збереження результатів 

In [22]:
output_excel = '../data/raw/result.xlsx'

df.to_excel(output_excel, index=False)

print(f"Результати кластеризації успішно збережені у файл: {output_excel}")

Результати кластеризації успішно збережені у файл: ../data/raw/result.xlsx
